In [196]:
from collections import namedtuple,deque
from heapq import heappush, heappop
from random import choice
from tqdm.auto import tqdm
import numpy as np

## Problem framing

In [197]:
PUZZLE_DIM = 7
action = namedtuple('Action', ['pos1', 'pos2'])

In [198]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [199]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing: 100%|██████████| 100000/100000 [00:00<00:00, 144633.62it/s]


array([[45, 33, 12, 17, 31, 38, 44],
       [46, 21, 24,  7, 27, 39, 22],
       [16,  8, 40, 10, 29, 36,  5],
       [48, 37, 32,  9, 13, 20, 41],
       [42, 11, 35,  1,  2, 47, 14],
       [ 6, 26, 34,  0, 25,  4, 18],
       [15,  3, 19, 28, 30, 43, 23]])

## Utility functions amd heuristics 
If we are able not to overestimate the estimation in a minimization problem, then the optimum is guaranteed, but the number of states evaluated increases, resulting in more time needed to find a result.
This is somehow acceptable for smaller instances (2x2 or 3x3), when reaching 4x4 or more, the optimum cannot be computed anymore if we want to do it in a reasonable time. Thus, we let on purpose the heuristic to overestimate (also a lot in case of 6x6 or 7x7), so that it is able to find a solution in a reasonable time.

In [200]:
def is_solved(state: np.ndarray) -> bool:
    return np.array_equal(state, np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM)))

class PuzzleHeuristicService:
    def __init__(self, goal_state: np.ndarray):
        self.goal_state = goal_state

    def heuristic_manhattan_distance(self, position: np.ndarray) -> int:
        distance = 0
        size = len(position)
        for i in range(size):
            for j in range(size):
                tile = position[i][j]
                if tile != 0:
                    target_row = (tile - 1) // size
                    target_col = (tile - 1) % size
                    distance += abs(i - target_row) + abs(j - target_col)
        return distance

    def heuristic_linear_conflict(self, position: np.ndarray) -> int:
        conflict = 0
        size = len(position)

        # Row conflicts
        for row in range(size):
            max_val = -1
            for col in range(size):
                value = position[row][col]
                if value != 0 and (value - 1) // size == row:
                    if value > max_val:
                        max_val = value
                    else:
                        conflict += 2

        # Column conflicts
        for col in range(size):
            max_val = -1
            for row in range(size):
                value = position[row][col]
                if value != 0 and (value - 1) % size == col:
                    if value > max_val:
                        max_val = value
                    else:
                        conflict += 2

        return conflict

    def heuristic_walking_distance(self, position: np.ndarray) -> int:
        # Calculate the Manhattan distance grid
        size = len(position)
        distance_grid = [[0] * size for _ in range(size)]

        for row in range(size):
            for col in range(size):
                value = position[row][col]
                if value != 0:
                    target_row = (value - 1) // size
                    target_col = (value - 1) % size
                    distance_grid[row][col] = abs(row - target_row) + abs(col - target_col)

        # Sum the distances
        walking_distance = sum(sum(row) for row in distance_grid)
        return walking_distance
    def compute_multiplication_factor(self) -> int:
        if PUZZLE_DIM <= 5:
            return 1
        elif PUZZLE_DIM >= 6:
            return 5

    def combined_heuristic(self, position: np.ndarray) -> int:
        if PUZZLE_DIM <= 3:
            return  self.heuristic_manhattan_distance(position)
        else:
            return self.compute_multiplication_factor() * (
                self.heuristic_manhattan_distance(position)
                + self.heuristic_linear_conflict(position)
                + self.heuristic_walking_distance(position)
            )


## A*

In [201]:
def solve_with_enhanced_a_star(initial_state: np.ndarray, goal_state: np.ndarray) -> tuple[list, int]:
    heuristic_service = PuzzleHeuristicService(goal_state)

    def calculate_heuristic(state: np.ndarray) -> int:
        return heuristic_service.combined_heuristic(state)

    # Priority queue: (f_score, g_score, state_bytes, path)
    open_set = []
    heappush(open_set, (calculate_heuristic(initial_state), 0, initial_state.tobytes(), []))
    visited = set()
    goal_state_bytes = goal_state.tobytes()

    counter_action_evaluated = 0

    while open_set:
        # Extract the node with the lowest f_score
        f_score, g_score, current_bytes, path = heappop(open_set)
        current_state = np.frombuffer(current_bytes, dtype=initial_state.dtype).reshape(initial_state.shape)

        # Check if we've reached the goal state
        if current_bytes == goal_state_bytes:
            return path, counter_action_evaluated

        # Add current state to visited
        visited.add(current_bytes)

        # Generate all possible moves
        for act in available_actions(current_state):
            counter_action_evaluated += 1
            next_state = do_action(current_state, act)
            next_bytes = next_state.tobytes()

            if next_bytes in visited:
                continue

            # Update scores
            new_g_score = g_score + 1
            new_f_score = new_g_score + calculate_heuristic(next_state)

            # Add new state to open set
            heappush(open_set, (new_f_score, new_g_score, next_bytes, path + [act]))

    return None, counter_action_evaluated  # No solution found

goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
path, evaluated_states = solve_with_enhanced_a_star(state, goal_state)
print("Path to solution:", path)
print("Number of states evaluated:", evaluated_states)
print("Goodness of the solution: "  + str(len(path)) + " moves")

Path to solution: [Action(pos1=(5, 3), pos2=(5, 2)), Action(pos1=(5, 2), pos2=(5, 1)), Action(pos1=(5, 1), pos2=(5, 0)), Action(pos1=(5, 0), pos2=(4, 0)), Action(pos1=(4, 0), pos2=(3, 0)), Action(pos1=(3, 0), pos2=(2, 0)), Action(pos1=(2, 0), pos2=(2, 1)), Action(pos1=(2, 1), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(1, 0)), Action(pos1=(1, 0), pos2=(2, 0)), Action(pos1=(2, 0), pos2=(3, 0)), Action(pos1=(3, 0), pos2=(3, 1)), Action(pos1=(3, 1), pos2=(4, 1)), Action(pos1=(4, 1), pos2=(5, 1)), Action(pos1=(5, 1), pos2=(5, 0)), Action(pos1=(5, 0), pos2=(4, 0)), Action(pos1=(4, 0), pos2=(3, 0)), Action(pos1=(3, 0), pos2=(3, 1)), Action(pos1=(3, 1), pos2=(4, 1)), Action(pos1=(4, 1), pos2=(4, 2)), Action(pos1=(4, 2), pos2=(4, 3)), Action(pos1=(4, 3), pos2=(4, 4)), Action(pos1=(4, 4), pos2=(5, 4)), Action(pos1=(5, 4), pos2=(5, 3)), Action(pos1=(5, 3), pos2=(5, 2)), Action(pos1=(5, 2), pos2=(5, 1)), Action(pos1=(5, 1), pos2=(5, 0)), Action(pos1=(5, 0), pos2=(4, 0)), Action(pos1=(4, 0), pos2=(4, 

In [202]:
current_state = state.copy()
for act in path:
    current_state = do_action(current_state, act)
print("Is the puzzle solved?", is_solved(current_state))
print(current_state)

Is the puzzle solved? True
[[ 1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14]
 [15 16 17 18 19 20 21]
 [22 23 24 25 26 27 28]
 [29 30 31 32 33 34 35]
 [36 37 38 39 40 41 42]
 [43 44 45 46 47 48  0]]
